### picking up metaphor/simile unique features

#### import libraries and data

In [26]:
import openpyxl
import pandas as pd

In [27]:
#sheet name lst
TITLE_LST = ["1_皮肉は短刀だ",
"2_夕日は銅貨だ",
"3_研究は登山だ",
"4_蝶は踊り子だ",
"5_時間は洪水だ",
"6_暴動は嵐だ",
"7_批判はメスだ",
"8_煙草は時限爆弾だ",
"9_真珠は水滴だ",
"10_つららは水晶細工（すいしょうざいく）だ",
"11_雨雲は薄墨だ",
"12_建物は角砂糖だ",
"13_衝撃は電気だ",
"14_学校は工場だ",
"15_子猫は王女だ",
"16_楽譜は暗号だ",
"17_煙突は石碑だ",
"18_空は鏡だ",
"19_子供は天使だ",
"20_柿はちょうちんだ",
"21_デモは雪崩（なだれ）だ",
"22_疑惑（ぎわく）は腫瘍（しゅよう）だ",
"23_教育は階段だ",
"24_微風は吐息だ",
"25_山は王様だ",
"26_仕事は牢獄（ろうごく）だ",
"27_愛は季節だ",
"28_利息は年貢だ",
"29_椿（つばき）はひとだまだ",
"30_審判は天秤だ",
"31_雑巾は亀だ",
"32_不安は濃霧（のうむ）だ",
"33_理論は建物だ",
"34_希望は灯火だ",
"35_運命は通り魔だ",
"36_沈黙は海底だ",
"37_霧（きり）はベールだ",
"38_激怒は噴火だ",
"39_眼は湖だ",
"40_唇（くちびる）は蛭（ひる）だ",
"41_笑顔は花だ",
"42_心臓は時計だ",
"43_麦畑は海だ",
"44_香水は花束だ",
"45_礼儀は檻（おり）だ"]

In [28]:
#set data directory
dir_metsimdiff_dat = "../data/MetSimDiff_All.xlsx"

#### pick unique features for one dataframe

In [29]:
#data sample
tmp_df = pd.read_excel(dir_metsimdiff_dat, sheet_name=TITLE_LST[0])#pick target sheet
tmp_df

,order,Feature,S_freq,M_freq,T_freq,V_freq,刺激候補,楠見先生ご提案,直喩候補,隠喩候補,共通候補,候補,楠見先生ご助言
0,3,切れる,7,3,0,8,NaN,NaN,1.0,NaN,NaN,直喩,NaN
1,5,怖い,5,3,3,5,NaN,NaN,1.0,NaN,NaN,直喩,NaN
2,7,差す-他動詞,4,3,0,7,NaN,NaN,1.0,NaN,NaN,直喩,NaN
3,4,差さる,6,2,0,0,NaN,NaN,1.0,NaN,NaN,直喩,NaN
4,1,鋭い,31,23,0,16,NaN,NaN,NaN,NaN,1.0,共通,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,96,折れる,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,97,嫌らしい,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,98,鬱陶しい,0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99,99,尖る,0,0,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
#pick data
def dat2unique_features(input_df):
    """
    以下の手順でFeatureにラベルを付与
    手順1. まずは, S_freqとM_freqのどちらも2未満の回答を除外
    手順2. 以下の条件でClassを付与
     - S_unique: 以下の条件のいずれかかが満たされる時、直喩に固有のfeature
         - S_freqが2以上で、M_freqが0
         - S_freqがM_freqの2倍以上（ただし、M_freqは1ではない）
     - M_unique: 以下の条件のいずれかかが満たされる時、隠喩に固有のfeature
         - M_freqが2以上で、S_freqが0
         - M_freqがS_freqの2倍以上（ただし、M_freqは1ではない）
     - Other: 上記のどちらにもあてはまらないもの
    """
    #不要列の削除
    input_df = input_df[["order", "Feature", "S_freq", "M_freq", "候補"]]
    
    #手順1に当てはまるデータの抽出
    step_one_dat = input_df.query('S_freq >= 2 or M_freq >= 2')

    #手順2のラベルを付与
    step_one_dat["Class"] = ""
    step_two_dat = step_one_dat.reset_index()

    #S_uniqueの付与
    for col, row in step_two_dat.iterrows():
        #S_uniqueの付与
        if (row["S_freq"] > 2 and row["M_freq"] == 0) or (row["S_freq"] >= row["M_freq"]*2 and row["M_freq"] != 1):
            step_two_dat.loc[col, "Class"] = "S_unique"
        #M_uniqueの付与
        elif (row["M_freq"] > 2 and row["S_freq"] == 0) or (row["M_freq"] >= row["S_freq"]*2 and row["S_freq"] != 1):
            step_two_dat.loc[col, "Class"] = "M_unique"
        #Otherの付与
        else:
            step_two_dat.loc[col, "Class"] = "Other"

    return step_two_dat

In [31]:
def unique_features2type_token_counts(unique_features_df, metaphor):
    """
    unique_featuresからtypeとtokensごとのカウントを求める
    """
    #データの初期化
    S_unique_type_count = 0
    S_unique_token_count = 0
    M_unique_type_count = 0
    M_unique_token_count = 0
    O_unique_type_count = 0
    O_unique_token_count = 0

    #数え上げ
    for col, row in unique_features_df.iterrows():
        if row["Class"] == "S_unique":
            S_unique_type_count += 1
            S_unique_token_count += row["S_freq"]
        elif row["Class"] == "M_unique":
            M_unique_type_count += 1
            M_unique_token_count += row["M_freq"]
        else:
            O_unique_type_count += 1
            O_unique_token_count += row["M_freq"]

    #データフレームに統合
    type_token_counts = pd.DataFrame([[metaphor, S_unique_type_count, S_unique_token_count, M_unique_type_count, M_unique_token_count, O_unique_type_count, O_unique_token_count]],
                                     columns=["metaphor", "S_unique_type_count", "S_unique_token_count", "M_unique_type_count", "M_unique_token_count", "O_unique_type_count", "O_unique_token_count"])

    return type_token_counts

#### pick unique features for all dataframe

In [77]:
#dat2unique_featuresで各ラベルを付与したエクセルを保存する
OUTPUT_EXCEL_DIR = "../result/MetSimDiff_All_with_class.xlsx"
book = openpyxl.Workbook()

for title in TITLE_LST:
    #現在のtitleのデータを読み込み
    cur_title_df = pd.read_excel(dir_metsimdiff_dat, sheet_name=title)#pick target sheet
    cur_unique_features_df = dat2unique_features(cur_title_df)

    #データをExcelに保存する
    active_sheet = book.create_sheet(title)
    active_sheet.title = title

    #1行目（見出し）の編集
    active_sheet.cell(column=1, row=1, value="index")
    active_sheet.cell(column=2, row=1, value="order")
    active_sheet.cell(column=3, row=1, value="Feature")
    active_sheet.cell(column=4, row=1, value="S_freq")
    active_sheet.cell(column=5, row=1, value="M_freq")
    active_sheet.cell(column=6, row=1, value="候補")
    active_sheet.cell(column=7, row=1, value="Class")

    #特徴と頻度の書き出し
    for row in range(len(cur_unique_features_df)):#行
        for column in range(7):#列
            active_sheet.cell(column=column+1, row=row+2, value=cur_unique_features_df.iloc[row, column])#値の書き込み

book.save(OUTPUT_EXCEL_DIR)

/var/folders/hk/52x26vr91258smgppkjhl_3w0000gn/T/ipykernel_3883/2535207488.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  step_one_dat["Class"] = ""
/var/folders/hk/52x26vr91258smgppkjhl_3w0000gn/T/ipykernel_3883/2535207488.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  step_one_dat["Class"] = ""
/var/folders/hk/52x26vr91258smgppkjhl_3w0000gn/T/ipykernel_3883/2535207488.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [52]:
#unique_features2type_token_countsを結合したデータフレームを作る
type_token_counts_df = pd.DataFrame(columns=["metaphor", "S_unique_type_count", "S_unique_token_count", "M_unique_type_count", "M_unique_token_count", "O_unique_type_count", "O_unique_token_count"])

for title in TITLE_LST:    
    #現在のtitleのデータを読み込み
    cur_title_df = pd.read_excel(dir_metsimdiff_dat, sheet_name=title)#pick target sheet
    cur_unique_features_df = dat2unique_features(cur_title_df)

    #countデータの作成
    tmp_df = unique_features2type_token_counts(cur_unique_features_df, title)

    #データフレームの結合
    type_token_counts_df = pd.concat([type_token_counts_df, tmp_df])

type_token_counts_ri = type_token_counts_df.reset_index()
type_token_counts_ri

/var/folders/hk/52x26vr91258smgppkjhl_3w0000gn/T/ipykernel_4727/2535207488.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  step_one_dat["Class"] = ""
/var/folders/hk/52x26vr91258smgppkjhl_3w0000gn/T/ipykernel_4727/2535207488.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  step_one_dat["Class"] = ""
/var/folders/hk/52x26vr91258smgppkjhl_3w0000gn/T/ipykernel_4727/2535207488.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

,index,metaphor,S_unique_type_count,S_unique_token_count,M_unique_type_count,M_unique_token_count,O_unique_type_count,O_unique_token_count
0,0,1_皮肉は短刀だ,2,13,7,16,13,67
1,0,2_夕日は銅貨だ,6,15,2,18,10,52
2,0,3_研究は登山だ,2,6,2,9,17,62
3,0,4_蝶は踊り子だ,3,11,1,2,19,85
4,0,5_時間は洪水だ,3,6,5,14,13,66
5,0,6_暴動は嵐だ,4,14,2,6,14,62
6,0,7_批判はメスだ,4,15,3,7,12,62
7,0,8_煙草は時限爆弾だ,4,9,2,5,8,51
8,0,9_真珠は水滴だ,2,5,8,41,10,55
9,0,10_つららは水晶細工（すいしょうざいく）だ,5,39,5,19,12,63


In [53]:
type_token_counts_ri.to_excel("../result/MetSimDiff_type_token_counts.xlsx")